Исходные данные:
1)Набор данных - https://www.kaggle.com/new-york-city/ny-2015-street-tree-census-tree-data?select=2015-street-tree-census-tree-data.csv

Задание:
1 Сколько в датасете объектов и признаков? Дать описание каждому признаку, если оно есть.

2 Сколько категориальных признаков, какие?

3 Столбец с макимальным количеством уникальных значений категориального признака?

4 Есть ли бинарные признаки?

5 Какие числовые признаки?

6 Есть ли пропуски?

7 Сколько объектов с пропусками?

8 Столбец с максимальным количеством пропусков?

9 Есть ли на ваш взгляд выбросы, аномальные значения?

10 Столбец с максимальным средним значением после нормировки признаков через стандартное отклонение?

11 Столбец с целевым признаком?

12 Сколько объектов попадает в тренировочную выборку при использовании train_test_split с параметрами test_size = 0.3, random_state = 42?

13 Между какими признаками наблюдается линейная зависимость (корреляция)?

14 Сколько признаков достаточно для объяснения 90% дисперсии после применения метода PCA?

15 Какой признак вносит наибольший вклад в первую компоненту?

Доп. задание: построить двухмерное представление данных с помощью алгоритма t-SNE. На сколько кластеров визуально на ваш взгляд разделяется выборка?

Установка необходимых библиотек:

In [ ]:
!pip install pandas
!pip install sklearn

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
data = pd.read_csv('2015-street-tree-census-tree-data.csv')

In [2]:
data.head()

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,...,boro_ct,state,latitude,longitude,x_sp,y_sp,council district,census tract,bin,bbl
0,180683,348711,2015-08-27T00:00:00.000,3,0,OnCurb,Alive,Fair,Acer rubrum,red maple,...,4073900,New York,40.723092,-73.844215,1027431.148,202756.7687,29.0,739.0,4052307.0,4.022210e+09
1,200540,315986,2015-09-03T00:00:00.000,21,0,OnCurb,Alive,Fair,Quercus palustris,pin oak,...,4097300,New York,40.794111,-73.818679,1034455.701,228644.8374,19.0,973.0,4101931.0,4.044750e+09
2,204026,218365,2015-09-05T00:00:00.000,3,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.717581,-73.936608,1001822.831,200716.8913,34.0,449.0,3338310.0,3.028870e+09
3,204337,217969,2015-09-05T00:00:00.000,10,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.713537,-73.934456,1002420.358,199244.2531,34.0,449.0,3338342.0,3.029250e+09
4,189565,223043,2015-08-30T00:00:00.000,21,0,OnCurb,Alive,Good,Tilia americana,American linden,...,3016500,New York,40.666778,-73.975979,990913.775,182202.4260,39.0,165.0,3025654.0,3.010850e+09


In [3]:
#исключим столбец с уникальными значениями: tree_id
data = data.drop('tree_id', axis = 1)

#исключим столбец, включающий результаты других столбцов
data = data.drop('problems', axis = 1)

#исключим столбец с индивидуальными параметрами
data = data.drop('created_at', axis = 1)

#исключим столбец с индивидуальными параметрами
data = data.drop('address', axis = 1)

#исключим столбцы с признаками, которые отсутствуют в документации
data = data.drop('council district', axis = 1)
data = data.drop('census tract', axis = 1)
data = data.drop('bin', axis = 1)
data = data.drop('bbl', axis = 1)

1)tree_id - уникальный идентификационный номер дерева (целое число)

2)block_id - Идентификационный номер блок-фасада, на который отображено дерево (целое число)

3)created_at - Дата нанесения деревьев на карту (дата)

4)tree_dbh - Диаметр на высоте груди дерева (целое число)

5)stump_diam - Диаметр пня (целое число)

6)curb_loc - Расположено ли дерево вдоль бордюра или со смещением от него (Текст Значения домена: OnCurb OffsetFromCurb)

7)status - Статус дерева (Текст Доменные значения: Живой, Пень, Мертвый)

8)health - Здоровье деревьев (Текст Доменные ценности: Хорошо, Неплохо, Плохо)

9)spc_latin - Научное / латинское название вида дерева (Текст)

10)spc_common - Общее название породы деревьев (Текст)

11)steward - Количество замеченных признаков управления (Текст Значения доменов: 1 или 2 ,3 или 4, 4 или больше, Ничего)

12)guards - Наличие и тип охраны деревьев (Текст Доменные ценности: Вредные Полезно Нет Не уверен)

13)sidewalk - Повреждение тротуара в непосредственной близости от дерева (Текст Значения домена: Повреждение, Не повреждено)

14)user_type - Категория пользователя, который собрал эту точку дерева (Текст Доменные ценности: Волонтер, TreesCount Персонал, Персонал NYC Parks)

15)root_stone - Проблемы с корнями, вызванные брусчаткой в ложе дерева (Текст Значения домена: Да Нет)

16)root_grate - Проблемы с корнями, вызванные металлическими решетками (Текст Значения домена: Да Нет)

17)root_other - Наличие других проблем с корнями (Текст Значения домена: Да Нет)

18)trunk_wire - Проблемы со стволом, вызванные тросом или проводами (Текст Значения домена: Да Нет)

19)trnk_light - Проблемы со стволом, вызванные освещением (Текст Значения домена: Да Нет)

20)trnk_other - Наличие других проблем со стволом (Текст Значения домена: Да Нет)

21)brch_light - Проблемы с ветками, вызванные освещением или проводами (Текст Значения домена: Да Нет)

22)brch_shoe - Проблемы с ветвями, вызванные обувью (Текст Значения домена: Да Нет)

23)brch_other - Наличие других проблем с ветвями (Текст Значения домена: Да Нет)

24)address - Предполагаемый уличный адрес дерева (Текст)

25)zipcode - Почтовый индекс (целое число)

26)zip_city - Город, полученный из почтового индекса (Текст)

27)cb_num - Общественный совет (Целое число)

28)borocode - Геокодирование на основе местоположения точки дерева (Целое число Доменные значения: 1 (Манхэттен) 2 (Бронкс) 3 (Бруклин) 4 (Квинс) 5 (Статен-Айленд))

29)boroname - Геокодирование на основе местоположения точки дерева (Текст Доменные ценности: Манхэттен Бронкс, Бруклин Квинс, Статен-Айленд)

30)cncldist - Округ совета Нью-Йорка (Целое число)

31)st_assem - Округ Ассамблеи штата Нью-Йорк (Целое число)

32)st_senate - Сенатский округ штата Нью-Йорк (Целое число)

33)nta - Код района табуляции соседей (Текст)

34)nta_name - Название зоны табуляции соседей (Текст)

35)boro_ct - Переписной участок (Текст)

36)state - Государство (Текст)

37)latitude - Широта точки дерева (Двойной)

38)longitude - Долгота точки дерева (Двойной)

39)x_sp - X координата точки дерева (Двойной)

40)y_sp - Y координата точки дерева (Двойной)

ВОПРОС 2. Сколько категориальных признаков, какие?

Всего 9 категориальных признаков
1)status Статус дерева
2)health Здоровье деревьев
3)steward Количество замеченных признаков управления
4)guards Наличие и тип охраны деревьев
5)user_type Категория пользователя, который собрал эту точку дерева
6)borocode Геокодирование на основе местоположения точки дерева
7)boroname Геокодирование на основе местоположения точки дерева

ВОПРОС 3. Столбец с макимальным количеством уникальных значений категориального признака?

In [4]:
ctgrl_priz = ['status','health','steward','guards','user_type','borocode','borough']

for column in data.columns:
    if column in ctgrl_priz:
        print(column, data[column].nunique())

status 3
health 3
steward 4
guards 4
user_type 3
borocode 5
borough 5


Максимальное число категориальных признаков у столбца borocode и borough

ВОПРОС 4. Есть ли бинарные признаки?

1)curb_loc 
2)sidewalk 
3)root_stone 
4)root_grate 
5)root_other 
6)trunk_wire 
7)trnk_light 
8)trnk_other 
9)brch_light 
10)brch_shoe 
11)brch_other

ВОПРОС 5. Какие числовые признаки?

1)tree_id 
2)block_id 
3)created_at
4)tree_dbh 
5)stump_diam 
6)zipcode 
7)cb_num 
8)cncldist 
9)st_assem 
10)st_senate 
11)latitude 
12)longitude 
13)x_sp 
14)y_sp 

ВОПРОС 6. Есть ли пропуски?

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 683788 entries, 0 to 683787
Data columns (total 37 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   block_id         683788 non-null  int64  
 1   tree_dbh         683788 non-null  int64  
 2   stump_diam       683788 non-null  int64  
 3   curb_loc         683788 non-null  object 
 4   status           683788 non-null  object 
 5   health           652172 non-null  object 
 6   spc_latin        652169 non-null  object 
 7   spc_common       652169 non-null  object 
 8   steward          652173 non-null  object 
 9   guards           652172 non-null  object 
 10  sidewalk         652172 non-null  object 
 11  user_type        683788 non-null  object 
 12  root_stone       683788 non-null  object 
 13  root_grate       683788 non-null  object 
 14  root_other       683788 non-null  object 
 15  trunk_wire       683788 non-null  object 
 16  trnk_light       683788 non-null  obje

ВОПРОС 7. Сколько объектов с пропусками?

Основываясь на результате info() - видим, что у некоторых объектов есть пропуски:
1)health
2)spc_latin
3)spc_common
4)steward
5)guards
6)sidewalk             

ВОПРОС 8. Столбец с максимальным количеством пропусков?

In [6]:
data.isnull().sum(axis = 0)

block_id               0
tree_dbh               0
stump_diam             0
curb_loc               0
status                 0
health             31616
spc_latin          31619
spc_common         31619
steward            31615
guards             31616
sidewalk           31616
user_type              0
root_stone             0
root_grate             0
root_other             0
trunk_wire             0
trnk_light             0
trnk_other             0
brch_light             0
brch_shoe              0
brch_other             0
postcode               0
zip_city               0
community board        0
borocode               0
borough                0
cncldist               0
st_assem               0
st_senate              0
nta                    0
nta_name               0
boro_ct                0
state                  0
latitude               0
longitude              0
x_sp                   0
y_sp                   0
dtype: int64

Следовательно, столбец/столбцы с наибольшим/им числом/ами пропусков - это spc_latin = 31619 и spc_common = 31619

ВОПРОС 9. Есть ли на ваш взгляд выбросы, аномальные значения?

In [7]:
for column in data.columns:
    print(data[column].value_counts())

204850    450
602362    358
208115    250
506756    206
233208    197
         ... 
341732      1
204966      1
202201      1
308948      1
316900      1
Name: block_id, Length: 101390, dtype: int64
4      60372
3      54454
2      41977
5      41642
11     37978
       ...  
75         1
151        1
154        1
159        1
450        1
Name: tree_dbh, Length: 146, dtype: int64
0      666134
4         966
5         939
3         779
6         754
        ...  
92          1
86          1
82          1
72          1
140         1
Name: stump_diam, Length: 100, dtype: int64
OnCurb            656896
OffsetFromCurb     26892
Name: curb_loc, dtype: int64
Alive    652173
Stump     17654
Dead      13961
Name: status, dtype: int64
Good    528850
Fair     96504
Poor     26818
Name: health, dtype: int64
Platanus x acerifolia                 87014
Gleditsia triacanthos var. inermis    64264
Pyrus calleryana                      58931
Quercus palustris                     53185
Acer platanoides

Аномальных значений и выбросов не замечено.

ВОПРОС 10. Столбец с максимальным средним значением после нормировки признаков через стандартное отклонение?

Нормализация через стандартное отклонение:

In [8]:
import math
arr = ['block_id','tree_dbh','stump_diam','postcode','community board','cncldist','st_assem','st_senate','boro_ct','latitude','longitude','x_sp','y_sp']
for column in arr:
    data[column] = (data[column] - data[column].mean())/(math.sqrt(data[column].var()))

Поиск максимального среднего

In [9]:
avrg = -math.inf
for column in arr:
    if data[column].mean() > avrg:
        avrg = data[column].mean()
print(avrg, column)

1.3488429021078534e-11 y_sp


Столбец с максимальным средним значением после нормирования - это y_sp

ВОПРОС 11. Столбец с целевым признаком?

Так как, мне надо выделить районы, в которых состояние деревьев аномально хорошее или плохое; 
выделить наилучший район по состоянию деревьев; 
выделить виды деревьев (или более общие элементы классификации), обладающие наилучшими показателями «здоровья», то будем оценивать столбцы:

1)status - Указывает, является ли дерево живым, стоящим мертвым или пнем.

2)zip_city - Город, полученный из почтового индекса

3)root_stone - Проблемы с корнями, вызванные брусчаткой в ложе дерева 

4)root_grate - Проблемы с корнями, вызванные металлическими решетками 

5)root_other - Наличие других проблем с корнями 

6)trunk_wire - Проблемы со стволом, вызванные тросом или проводами 

7)trnk_light - Проблемы со стволом, вызванные освещением 

8)trnk_other - Наличие других проблем со стволом 

9)brch_light - Проблемы с ветками, вызванные освещением или проводами

10)brch_shoe - Проблемы с ветвями, вызванные обувью 

11)brch_other - Наличие других проблем с ветвями 

12)spc_common - Общее название породы деревьев

ВОПРОС 12. Сколько объектов попадает в тренировочную выборку при использовании train_test_split с параметрами test_size = 0.3, random_state = 42?

Обработка категориальных признаков и бинарных:

In [10]:
data['state'] = np.where(data['state'] == 'New York', 1, 0)

data.loc[data.spc_latin == 'NaN', 'spc_latin'] = 0
data.loc[data.spc_common == 'NaN', 'spc_common'] = 0
data.loc[data.steward == 'NaN', 'steward'] = 0
data.loc[data.guards == 'NaN', 'guards'] = 0
data.loc[data.sidewalk == 'NaN', 'sidewalk'] = 0


#Расположено ли дерево вдоль бордюра или со смещением от него
data['curb_loc'] = np.where(data['curb_loc'] == 'OnCurb', 1, 0)

#Повреждение тратуара в непосредственной близости от дерева
data['sidewalk'] = np.where(data['sidewalk'] == 'Damage', 1, 0)

#Указывает на наличие проблемы с корнями, вызванной брусчаткой в ложе дерева
data['root_stone'] = np.where(data['root_stone'] == 'Yes', 1, 0)

#Указывает на наличие проблемы с корнями, вызванной металлическими решетками в ложе дерева
data['root_grate'] = np.where(data['root_grate'] == 'Yes', 1, 0)

#Указывает на наличие других корневых проблем
data['root_other'] = np.where(data['root_other'] == 'Yes', 1, 0)

#Указывает на наличие проблемы со стволом, вызванной проволокой или веревкой, обмотанной вокруг ствола
data['trunk_wire'] = np.where(data['trunk_wire'] == 'Yes', 1, 0)

#Указывает на наличие проблем со стволом, вызванных установленным на дереве освещением
data['trnk_light'] = np.where(data['trnk_light'] == 'Yes', 1, 0)

#Указывает на наличие других проблем с магистралью
data['trnk_other'] = np.where(data['trnk_other'] == 'Yes', 1, 0)

#Указывает на наличие проблемы с ветвями, вызванной светильниками (обычно струнными) или проводами в ветвях
data['brch_light'] = np.where(data['brch_light'] == 'Yes', 1, 0)

#Указывает на наличие проблемы с ветками, вызванной кроссовками в ветках
data['brch_shoe'] = np.where(data['brch_shoe'] == 'Yes', 1, 0)

#Указывает на наличие других проблем с ветвями
data['brch_other'] = np.where(data['brch_other'] == 'Yes', 1, 0)

#Город, полученный из почтового индекса
my_set = set(data.zip_city)
i = 0
for item in my_set:
    data['zip_city'] = data['zip_city'].replace(item, i)
    i = i + 1

#Это название NTA, соответствующее области табуляции района по переписи населения США 2010 года, в которую попадает точка дерева
my_set = set(data.nta_name)
i = 0
for item in my_set:
    data['nta_name'] = data['nta_name'].replace(item, i)
    i = i + 1

#Это код NTA, соответствующий району, в который попадает точка дерева, согласно переписи населения США 2010 года.
my_set = set(data.nta)
i = 0
for item in my_set:
    data['nta'] = data['nta'].replace(item, i)
    i = i + 1

#Название района, в котором находится точка произрастания деревьев
my_set = set(data.borough)
i = 0
for item in my_set:
    data['borough'] = data['borough'].replace(item, i)
    i = i + 1

#Город, полученный из почтового индекса. Часто (но не всегда) это то же самое, что и район.
my_set = set(data.postcode)
i = 0
for item in my_set:
    data['postcode'] = data['postcode'].replace(item, i)
    i = i + 1

#Указывает, присутствует ли ограждение, и считает ли пользователь, что это полезное или вредное ограждение. Не регистрируется для мертвых деревьев и пней.
my_set = set(data.guards)
i = 0
for item in my_set:
    data['guards'] = data['guards'].replace(item, i)
    i = i + 1

#Это поле описывает категорию пользователя, который собрал данные этой точки дерева.
my_set = set(data.user_type)
i = 0
for item in my_set:
    data['user_type'] = data['user_type'].replace(item, i)
    i = i + 1    
    
#Указывает количество уникальных признаков бережного отношения, отмеченных для данного дерева. Не регистрируется для пней или мертвых деревьев.
my_set = set(data.steward)
i = 0
for item in my_set:
    data['steward'] = data['steward'].replace(item, i)
    i = i + 1

#Общее название вида, например, "красный клен"
my_set = set(data.spc_common)
i = 0
for item in my_set:
    data['spc_common'] = data['spc_common'].replace(item, i)
    i = i + 1
    
#Научное название вида, например, "Acer rubrum".
my_set = set(data.spc_latin)
i = 0
for item in my_set:
    data['spc_latin'] = data['spc_latin'].replace(item, i)
    i = i + 1

#Указывает, является ли дерево живым, стоящим мертвым или пнем.
data.loc[data.status == 'Alive', 'status'] = 2
data.loc[data.status == 'Stump', 'status'] = 1
data.loc[data.status == 'Dead', 'status'] = 0

#Здоровье деревьев
data.loc[data.health == 'Good', 'health'] = 2
data.loc[data.health == 'Fair', 'health'] = 1
data.loc[data.health == 'Poor', 'health'] = 0
data.loc[data.health == 'NaN', 'health'] = 0

Исключаем столбец с целевыми признаками из датасета для тренировки, а также столбы с признаками, которые мы обработали строчкой выше.

In [11]:
data_training = data.drop(['health',
                           'borough',
                           'root_stone',
                           'root_grate',
                           'root_other',
                           'trunk_wire',
                           'trnk_light',
                           'trnk_other',
                           'brch_light',
                           'brch_shoe',
                           'zip_city',
                           'brch_other',
                           'status',
                           'spc_common'],
                          axis = 1)
data_target = data[['health',
                           'borough',
                           'root_stone',
                           'root_grate',
                           'root_other',
                           'trunk_wire',
                           'trnk_light',
                           'trnk_other',
                           'brch_light',
                           'brch_shoe',
                           'zip_city',
                           'brch_other',
                           'status',
                           'spc_common']]

In [12]:
data_training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 683788 entries, 0 to 683787
Data columns (total 23 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   block_id         683788 non-null  float64
 1   tree_dbh         683788 non-null  float64
 2   stump_diam       683788 non-null  float64
 3   curb_loc         683788 non-null  int32  
 4   spc_latin        683788 non-null  int64  
 5   steward          683788 non-null  int64  
 6   guards           683788 non-null  int64  
 7   sidewalk         683788 non-null  int32  
 8   user_type        683788 non-null  int64  
 9   postcode         683788 non-null  float64
 10  community board  683788 non-null  float64
 11  borocode         683788 non-null  int64  
 12  cncldist         683788 non-null  float64
 13  st_assem         683788 non-null  float64
 14  st_senate        683788 non-null  float64
 15  nta              683788 non-null  int64  
 16  nta_name         683788 non-null  int6

Разделим данные на тренировочные и тестовые

In [13]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data_training, data_target, test_size=.3, random_state=42)

In [14]:
x_train

,block_id,tree_dbh,stump_diam,curb_loc,spc_latin,steward,guards,sidewalk,user_type,postcode,...,st_assem,st_senate,nta,nta_name,boro_ct,state,latitude,longitude,x_sp,y_sp
160056,1.727696,2.260704,-0.131438,1,72,2,1,0,2,12.0,...,1.539999,1.810919,94,8,-1.168430,1,1.737253,0.695695,0.693014,1.737568
287015,0.820626,-0.146713,-0.131438,1,39,2,1,0,2,97.0,...,0.643682,0.457893,50,178,1.372428,1,-1.090468,-1.652555,-1.652426,-1.090721
113753,0.857182,-1.063825,-0.131438,1,14,2,1,0,1,74.0,...,0.590958,0.457893,95,130,1.375743,1,-1.901406,-2.481750,-2.484449,-1.899898
423584,-0.044332,-1.293102,5.339286,1,0,0,0,0,1,100.0,...,-0.674430,-0.759830,150,128,0.506765,1,-0.143172,0.537737,0.538447,-0.143219
238843,0.812397,-0.261352,-0.131438,1,61,2,4,0,2,1.0,...,0.696407,0.322591,71,132,1.362816,1,-1.211493,-1.162457,-1.161957,-1.212322
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259178,1.711055,0.655759,-0.131438,1,59,2,1,1,2,170.0,...,1.909070,1.540314,168,75,-1.177445,1,1.545682,0.599417,0.597330,1.545810
365838,-0.092243,-0.719908,-0.131438,1,117,1,3,0,0,156.0,...,-1.096226,-0.759830,73,167,0.526750,1,0.418480,0.204343,0.204545,0.417969
131932,1.738215,-0.834547,-0.131438,1,76,2,1,0,2,12.0,...,1.698172,2.081524,182,66,-1.155502,1,1.875252,0.717991,0.714998,1.875619
671155,1.670677,0.197203,-0.131438,1,52,2,1,0,0,78.0,...,1.434550,1.810919,4,89,-1.161712,1,1.735965,0.322483,0.320883,1.735666


В тренировочную выборку попадет 478651 объектов

ВОПРОС 13. Между какими признаками наблюдается линейная зависимость (корреляция)?

Проверим коэффициент VIF (Variable Inflation Factors) для всех признаков. Если VIF > 10, то есть четкая линейная зависимость.

In [15]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif_data = pd.DataFrame()
vif_data["column_name"] = data_training.columns
vif_data["VIF"] = [variance_inflation_factor(data_training.values, i) for i in range(len(data_training.columns))]

Рассмотрим значения с VIF > 10

In [16]:
vif_data.loc[vif_data['VIF'] > 10.0]

,column_name,VIF
10,community board,2.026474e+03
11,borocode,3.109439e+03
12,cncldist,1.272854e+01
13,st_assem,1.138028e+01
14,st_senate,1.207605e+01
17,boro_ct,1.293885e+03
18,state,2.588884e+04
19,latitude,7.280533e+06
20,longitude,3.253131e+06
21,x_sp,3.244514e+06


VIF > 10 у: 
community board,

borocode,

cncldist,

st_assem,

st_senate,

boro_ct,

latitude,

longitude,

x_sp,

y_sp

ВОПРОС 14. Сколько признаков достаточно для объяснения 90% дисперсии после применения метода PCA?

Сначала стандартизируем данные:

In [17]:
from sklearn import preprocessing
std_data = pd.DataFrame(preprocessing.scale(x_train),columns = x_train.columns)

Теперь используем PCA, чтобы он объяснил 90% дисперсии:

In [18]:
from sklearn.decomposition import PCA

for i in range(1, len(std_data.columns)):
    pca = PCA(n_components=i)
    pca.fit(std_data)
    print(i, sum(pca.explained_variance_ratio_))

1 0.2622238186142132
2 0.415635020069259
3 0.49975056028924564
4 0.5775101048526545
5 0.6304893433862014
6 0.6823983053906328
7 0.7292451547224947
8 0.7726457413803577
9 0.8119331320096527
10 0.8489594587832856
11 0.8838136131753305
12 0.9160502416579677
13 0.9454249731455419
14 0.9744873856569652
15 0.9888423611280118
16 0.9941414272661052
17 0.9971929785579058
18 0.9999593456748138
19 0.9999880338862963
20 0.9999999840528653
21 0.999999997431174
22 1.0


Для объяснения 90% дисперсии - достаточно 11 признаков

Вопрос 15. Какой признак вносит наибольший вклад в первую компоненту?

In [19]:
pca = PCA(n_components=11)
x = pca.fit(std_data)

first_component = pca.components_[0]
i = 0
answ = first_component[i]

for j in range(0, len(first_component)):
    if  abs(first_component[j]) >  abs(answ):
        answ = first_component[j]
        i = j
print(i, answ)

10 -0.38488364900761773


Наибольший вклад в первую компоненту вносит 11-ый признак.

In [20]:
std_data.iloc[:, 11]

0        -1.163300
1         1.407915
2         1.407915
3         0.550844
4         1.407915
            ...   
478646   -1.163300
478647    0.550844
478648   -1.163300
478649   -1.163300
478650   -2.020372
Name: borocode, Length: 478651, dtype: float64

Это признак borocode.